In [11]:
# Importation des librairies
import json
import os
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold, cross_validate

In [12]:
pd.set_option('future.no_silent_downcasting', True)

In [13]:
def read_csv_files_to_dict(base_path, dimension, file_prefix, missing_percentages, strategies=None):
    dataframes_dict = {}

    for percentage in missing_percentages:
        percentage_key = f"{int(percentage * 100)}%"

        if dimension == 'Completeness':
            dataframes_dict[percentage_key] = {}

            if strategies is not None:
                for strategy_name, strategy_func, subfolder in strategies:
                    file_path = f"{base_path}/{dimension}/{subfolder}/{file_prefix}_{percentage_key}.csv"

                    try:
                        df = pd.read_csv(file_path)
                        dataframes_dict[percentage_key][strategy_name] = df
                    except FileNotFoundError:
                        print(f"File not found: {file_path}")
                        dataframes_dict[percentage_key][strategy_name] = None
            else:
                print("No strategies provided for Completeness dimension.")
        elif dimension == 'Unicity':
            file_path = f"{base_path}/{dimension}/{file_prefix}_{percentage_key}_2x.csv"
            try:
                df = pd.read_csv(file_path)
                dataframes_dict[percentage_key] = df
            except FileNotFoundError:
                print(f"File not found: {file_path}")
                dataframes_dict[percentage_key] = None
        else:
            file_path = f"{base_path}/{dimension}/{file_prefix}_{percentage_key}.csv"
            try:
                df = pd.read_csv(file_path)
                dataframes_dict[percentage_key] = df
            except FileNotFoundError:
                print(f"File not found: {file_path}")
                dataframes_dict[percentage_key] = None

    return dataframes_dict

def update_json_results(output_path, model_name, pollution_percentage, results):
    # Charger le fichier JSON existant, ou initialiser une nouvelle structure si le fichier n'existe pas
    if os.path.exists(output_path):
        with open(output_path, 'r') as json_file:
            results_dict = json.load(json_file)
    else:
        results_dict = {
            "models": []
        }

    # Trouver ou ajouter l'entrée pour le modèle spécifié
    model_entry = next((model for model in results_dict["models"] if model["model"] == model_name), None)

    if not model_entry:
        model_entry = {
            "model": model_name,
            "pollution_metrics": []
        }
        results_dict["models"].append(model_entry)

    # Chercher si le pourcentage de pollution existe déjà pour ce modèle
    existing_entry = next((item for item in model_entry["pollution_metrics"] if item["pollution_percentage"] == pollution_percentage), None)

    if existing_entry:
        # Si le pourcentage de pollution existe, remplacer les métriques
        existing_entry["metrics"] = results
    else:
        # Sinon, ajouter une nouvelle entrée pour ce pourcentage
        model_entry["pollution_metrics"].append({
            "pollution_percentage": pollution_percentage,
            "metrics": results
        })

    # Écrire les résultats mis à jour dans le fichier JSON
    with open(output_path, 'w') as json_file:
        json.dump(results_dict, json_file, indent=4)

    print(f"Results saved to {output_path}")

def update_json_results_completeness(output_path, model_name, strategy, pollution_percentage, results):
    # Charger le fichier JSON existant, ou initialiser une nouvelle structure si le fichier n'existe pas
    if os.path.exists(output_path):
        with open(output_path, 'r') as json_file:
            results_dict = json.load(json_file)
    else:
        results_dict = {
            "models": [
                {
                    "model": model_name,
                    "imputation_strategies": []
                }
            ]
        }

    # Trouver ou ajouter l'entrée pour le modèle spécifié
    model_entry = next((model for model in results_dict["models"] if model["model"] == model_name), None)

    if not model_entry:
        model_entry = {
            "model": model_name,
            "imputation_strategies": []
        }
        results_dict["models"].append(model_entry)

    # Trouver ou créer l'entrée pour la stratégie d'imputation actuelle
    strategy_entry = next((item for item in model_entry["imputation_strategies"] if item["strategy"] == strategy), None)

    if not strategy_entry:
        # Ajouter une nouvelle stratégie d'imputation si elle n'existe pas encore
        strategy_entry = {
            "strategy": strategy,
            "pollution_metrics": []
        }
        model_entry["imputation_strategies"].append(strategy_entry)

    # Vérifier si le pourcentage de pollution existe déjà pour cette stratégie
    existing_entry = next((item for item in strategy_entry["pollution_metrics"] if item["pollution_percentage"] == pollution_percentage), None)

    if existing_entry:
        # Si le pourcentage de pollution existe, remplacer les métriques
        existing_entry["metrics"] = results
    else:
        # Sinon, ajouter une nouvelle entrée pour ce pourcentage
        strategy_entry["pollution_metrics"].append({
            "pollution_percentage": pollution_percentage,
            "metrics": results
        })

    # Écrire les résultats mis à jour dans le fichier JSON
    with open(output_path, 'w') as json_file:
        json.dump(results_dict, json_file, indent=4)

    print(f"Results saved to {output_path}")

def clean_column_names(df):
    df.columns = df.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)
    return df

def prepare_heart_data(heart_df):
    # Remplacer les valeurs de la colonne 'HadHeartAttack' par des booléens
    heart_df['HadHeartAttack'] = heart_df['HadHeartAttack'].replace({'Yes': True, 'No': False}).astype(bool)

    # Convertir les variables catégorielles en variables dummy, en supprimant la première catégorie
    heart_df = pd.get_dummies(heart_df, drop_first=True)

    heart_df = clean_column_names(heart_df)
    # Séparer les variables indépendantes (X) de la variable cible (y)
    X = heart_df.drop('HadHeartAttack', axis=1)
    y = heart_df['HadHeartAttack']

    return X, y

def train_and_evaluate(X, y):
    model = LGBMClassifier(random_state=42)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

    cv_results = cross_validate(model, X, y, cv=kfold, scoring=scoring)

    results = {
        "accuracy": cv_results['test_accuracy'].mean(),
        "precision": cv_results['test_precision_macro'].mean(),
        "recall": cv_results['test_recall_macro'].mean(),
        "f1_score": cv_results['test_f1_macro'].mean()
    }

    return results

## Completeness

In [14]:
# Utilisation de la fonction dans votre code existant
missing_percentages = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

# Strategy :
strategy_delete = {
    'numerical': 'delete',
    'categorical': 'delete'
}

strategy_mean_mode = {
    'numerical': 'mean',
    'categorical': 'mode'
}

strategy_median_new = {
    'numerical': 'median',
    'categorical': 'new'
}

strategy_decision_tree = {
    'numerical': 'decision_tree',
    'categorical': 'decision_tree'
}

strategy_mean_new = {
    'numerical': 'mean',
    'categorical': 'new'
}

strategy_knn_mode = {
    'numerical': 'knn',
    'categorical': 'mode'
}

strategies = [
    ("Mean and Mode", strategy_mean_mode, "Mean and Mode"),
    ("Median and New", strategy_median_new, "Median and New"),
    ("Decision Tree", strategy_decision_tree, "Decision Tree"),
    ("Mean and New", strategy_mean_new, "Mean and New")
]

heart_df_dict = read_csv_files_to_dict('../../Data/Classification/Indicators of Heart Disease', 'Completeness', 'heart', missing_percentages, strategies)

# Chemin du fichier JSON où les résultats seront enregistrés
output_path = "../../Results/Heart Disease/Completeness.json"

# Boucle sur les DataFrames pour chaque pourcentage de pollution
for key, heart_df_strategies in heart_df_dict.items():
    for strategy_name, heart_df in heart_df_strategies.items():
        X, y = prepare_heart_data(heart_df)

        results = train_and_evaluate(X, y)

        # Extraire le pourcentage de pollution à partir de la clé
        pollution_percentage = float(key.replace('%', ''))

        # Mise à jour des résultats dans le fichier JSON en utilisant la fonction
        update_json_results_completeness(output_path, "LightGBM", strategy_name, pollution_percentage, results)

[LightGBM] [Info] Number of positive: 10748, number of negative: 186069
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011628 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1158
[LightGBM] [Info] Number of data points in the train set: 196817, number of used features: 122
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.054609 -> initscore=-2.851398
[LightGBM] [Info] Start training from score -2.851398
[LightGBM] [Info] Number of positive: 10748, number of negative: 186069
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011632 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1162
[LightGBM] [Info] Number of data points in the train set: 196817, number of used features: 122
[LightGBM] [

## Consistent Representation

In [15]:
# Utilisation de la fonction dans votre code existant
missing_percentages = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
heart_df_dict = read_csv_files_to_dict('../../Data/Classification/Indicators of Heart Disease', 'Consistent Representation', 'heart', missing_percentages)

# Chemin du fichier JSON où les résultats seront enregistrés
output_path = "../../Results/Heart Disease/Consistent Representation.json"

# Boucle sur les DataFrames pour chaque pourcentage de pollution
for key, heart_df in heart_df_dict.items():
    X, y = prepare_heart_data(heart_df)

    results = train_and_evaluate(X, y)

    # Extraire le pourcentage de pollution à partir de la clé
    pollution_percentage = float(key.replace('%', ''))

    # Mise à jour des résultats dans le fichier JSON
    update_json_results(output_path, "LightGBM", pollution_percentage, results)

[LightGBM] [Info] Number of positive: 10748, number of negative: 186069
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011526 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 903
[LightGBM] [Info] Number of data points in the train set: 196817, number of used features: 121
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.054609 -> initscore=-2.851398
[LightGBM] [Info] Start training from score -2.851398
[LightGBM] [Info] Number of positive: 10748, number of negative: 186069
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012773 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 907
[LightGBM] [Info] Number of data points in the train set: 196817, number of used features: 121
[LightGBM] [In

## Feature Accuracy

In [16]:
# Utilisation de la fonction dans votre code existant
missing_percentages = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
heart_df_dict = read_csv_files_to_dict('../../Data/Classification/Indicators of Heart Disease', 'Feature Accuracy', 'heart', missing_percentages)

# Chemin du fichier JSON où les résultats seront enregistrés
output_path = "../../Results/Heart Disease/Feature Accuracy.json"

# Boucle sur les DataFrames pour chaque pourcentage de pollution
for key, heart_df in heart_df_dict.items():
    X, y = prepare_heart_data(heart_df)

    results = train_and_evaluate(X, y)

    # Extraire le pourcentage de pollution à partir de la clé
    pollution_percentage = float(key.replace('%', ''))

    # Mise à jour des résultats dans le fichier JSON
    update_json_results(output_path, "LightGBM", pollution_percentage, results)

[LightGBM] [Info] Number of positive: 10748, number of negative: 186069
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012604 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 903
[LightGBM] [Info] Number of data points in the train set: 196817, number of used features: 121
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.054609 -> initscore=-2.851398
[LightGBM] [Info] Start training from score -2.851398
[LightGBM] [Info] Number of positive: 10748, number of negative: 186069
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012413 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 907
[LightGBM] [Info] Number of data points in the train set: 196817, number of used features: 121
[LightGBM] [In

/opt/anaconda3/envs/Impact-of-data-quality-on-machine-learning-models/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[LightGBM] [Info] Number of positive: 10748, number of negative: 186070
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002001 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1076
[LightGBM] [Info] Number of data points in the train set: 196818, number of used features: 121
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.054609 -> initscore=-2.851403
[LightGBM] [Info] Start training from score -2.851403
[LightGBM] [Info] Number of positive: 10748, number of negative: 186070
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002573 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1075
[LightGBM] [Info] Number of data points in the train set: 196818, number of used features: 121
[LightGBM] [

/opt/anaconda3/envs/Impact-of-data-quality-on-machine-learning-models/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[LightGBM] [Info] Number of positive: 10748, number of negative: 186069
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002066 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1075
[LightGBM] [Info] Number of data points in the train set: 196817, number of used features: 121
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.054609 -> initscore=-2.851398
[LightGBM] [Info] Start training from score -2.851398
[LightGBM] [Info] Number of positive: 10748, number of negative: 186070
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001656 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1076
[LightGBM] [Info] Number of data points in the train set: 196818, number of used features: 121
[LightGBM] [

/opt/anaconda3/envs/Impact-of-data-quality-on-machine-learning-models/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[LightGBM] [Info] Number of positive: 10748, number of negative: 186070
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001790 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1074
[LightGBM] [Info] Number of data points in the train set: 196818, number of used features: 121
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.054609 -> initscore=-2.851403
[LightGBM] [Info] Start training from score -2.851403
Results saved to ../../Results/Heart Disease/Feature Accuracy.json
[LightGBM] [Info] Number of positive: 10748, number of negative: 186069
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001455 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1080
[LightGBM] [Info] Number of data points 

## Target Accuracy

In [17]:
pollution_percentage_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
heart_df_dict = read_csv_files_to_dict('../../Data/Classification/Indicators of Heart Disease', 'Target Accuracy', 'heart', pollution_percentage_levels)

# Chemin du fichier JSON où les résultats seront enregistrés
output_path = "../../Results/Heart Disease/Target Accuracy.json"

# Boucle sur les DataFrames pour chaque pourcentage de pollution
for key, heart_df in heart_df_dict.items():
    X, y = prepare_heart_data(heart_df)

    results = train_and_evaluate(X, y)

    # Extraire le pourcentage de pollution à partir de la clé
    pollution_percentage = float(key.replace('%', ''))

    # Mise à jour des résultats dans le fichier JSON
    update_json_results(output_path, "LightGBM", pollution_percentage, results)

[LightGBM] [Info] Number of positive: 10748, number of negative: 186069
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009925 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 903
[LightGBM] [Info] Number of data points in the train set: 196817, number of used features: 121
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.054609 -> initscore=-2.851398
[LightGBM] [Info] Start training from score -2.851398
[LightGBM] [Info] Number of positive: 10748, number of negative: 186069
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011104 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 907
[LightGBM] [Info] Number of data points in the train set: 196817, number of used features: 121
[LightGBM] [In

## Target Class Balance

In [18]:
balance_levels = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
heart_df_dict = read_csv_files_to_dict('../../Data/Classification/Indicators of Heart Disease', 'Target Class Balance', 'heart', balance_levels)

# Chemin du fichier JSON où les résultats seront enregistrés
output_path = "../../Results/Heart Disease/Target Class Balance.json"

# Boucle sur les DataFrames pour chaque pourcentage de pollution
for key, heart_df in heart_df_dict.items():
    X, y = prepare_heart_data(heart_df)

    results = train_and_evaluate(X, y)

    # Extraire le pourcentage de pollution à partir de la clé
    pollution_percentage = float(key.replace('%', ''))

    # Mise à jour des résultats dans le fichier JSON
    update_json_results(output_path, "LightGBM", pollution_percentage, results)

[LightGBM] [Info] Number of positive: 178210, number of negative: 18606
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012306 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 870
[LightGBM] [Info] Number of data points in the train set: 196816, number of used features: 121
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.905465 -> initscore=2.259479
[LightGBM] [Info] Start training from score 2.259479
[LightGBM] [Info] Number of positive: 178211, number of negative: 18606
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010181 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 868
[LightGBM] [Info] Number of data points in the train set: 196817, number of used features: 121
[LightGBM] [Info

## Unicity

In [19]:
pollution_percentage_levels = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
heart_df_dict = read_csv_files_to_dict('../../Data/Classification/Indicators of Heart Disease', 'Unicity', 'heart', pollution_percentage_levels)

# Chemin du fichier JSON où les résultats seront enregistrés
output_path = "../../Results/Heart Disease/Unicity.json"

# Boucle sur les DataFrames pour chaque pourcentage de pollution
for key, heart_df in heart_df_dict.items():
    X, y = prepare_heart_data(heart_df)

    results = train_and_evaluate(X, y)

    # Extraire le pourcentage de pollution à partir de la clé
    pollution_percentage = float(key.replace('%', ''))

    # Mise à jour des résultats dans le fichier JSON
    update_json_results(output_path, "LightGBM", pollution_percentage, results)

[LightGBM] [Info] Number of positive: 10748, number of negative: 186069
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012062 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 903
[LightGBM] [Info] Number of data points in the train set: 196817, number of used features: 121
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.054609 -> initscore=-2.851398
[LightGBM] [Info] Start training from score -2.851398
[LightGBM] [Info] Number of positive: 10748, number of negative: 186069
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023208 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 907
[LightGBM] [Info] Number of data points in the train set: 196817, number of used features: 121
[LightGBM] [In